In [37]:
import pandas as pd
import numpy as np
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)
        self.z = arr.iloc[0, :-1].values.astype(np.float64)  # Initial feasible point (shape (n,))
        self.c = arr.iloc[1, :-1].values.astype(np.float64)  # Cost vector (shape (n,))
        self.b = arr.iloc[2:, -1].values.astype(np.float64)  # Constraint vector (shape (m,))
        self.A = arr.iloc[2:, :-1].values.astype(np.float64) # Constraint coefficients matrix (shape (m, n))
        self.n = self.A.shape[1]  # Number of variables
        self.m = self.A.shape[0]  # Number of constraints

        if self.m < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")

        self.max_iterations = 1000  # Increased limit for iterations
        self.tolerance = 1e-8  # Numerical tolerance for feasibility and degeneracy checks
        self.initial_perturbation_scale = 1e-4  # Initial perturbation scale
        self.epsilon = self.initial_perturbation_scale  # Current perturbation scale

        # List to store sequence of vertices visited
        self.visited_vertices = []
        self.visited_set = set()  # To detect cycling

        # Perturb b initially to handle degeneracy
        self.perturb_b()

    def perturb_b(self, scale=None):
        """Perturb the constraint vector b slightly to handle degeneracy.
           Each constraint is perturbed by a different power of epsilon."""
        if scale is None:
            scale = self.epsilon
        # Apply perturbation as epsilon^i for the i-th constraint
        perturbations = scale ** (np.arange(1, self.m + 1))
        self.b += perturbations

    def isFeasible(self):
        """Check if the solution z is feasible."""
        return np.all(np.matmul(self.A, self.z) <= self.b + self.tolerance)

    def __activeAndUnActiveConst(self):
        """Classify active and inactive constraints based on the current z."""
        constraint_values = np.matmul(self.A, self.z)
        active_indices = np.where(np.abs(constraint_values - self.b) <= self.tolerance)[0]
        inactive_indices = np.where(constraint_values < self.b - self.tolerance)[0]

        self.A1 = self.A[active_indices]
        self.B1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.B2 = self.b[inactive_indices]

    def __isVertex(self):
        """Check if the current solution z is a vertex."""
        self.__activeAndUnActiveConst()
        return self.A1.shape[0] == self.n

    def find_initial_vertex(self):
        """Find an initial vertex for the feasible region."""
        if not self.isFeasible():
            raise ValueError("Initial point z is not feasible.")

        iteration = 0
        while not self.__isVertex():
            if iteration >= self.max_iterations:
                raise ValueError("Max iterations reached while finding initial vertex.")
            iteration += 1

            self.__activeAndUnActiveConst()
            A1 = self.A1
            A2 = self.A2
            b2 = self.B2

            if A1.size == 0:
                # No active constraints, move towards the nearest constraint
                slack = self.b - np.dot(self.A, self.z)
                min_slack_index = np.argmin(slack)
                direction = self.A[min_slack_index].astype(np.float64)
                direction /= np.linalg.norm(direction)
            else:
                nullspace = np.array(Matrix(A1).nullspace())
                if nullspace.size == 0:
                    break  # At a vertex

                direction = nullspace[0].astype(np.float64).flatten()
                norm = np.linalg.norm(direction)
                if norm == 0:
                    raise ValueError("Zero direction vector encountered.")
                direction /= norm

            temp = np.dot(A2, direction)
            temp = np.where(np.abs(temp) < self.tolerance, self.tolerance, temp)
            alpha_values = (b2 - np.dot(A2, self.z)) / temp
            positive_alphas = [alpha for alpha in alpha_values if alpha > self.tolerance]
            if not positive_alphas:
                raise ValueError("Cannot find a positive alpha to move towards a vertex.")

            alpha = min(positive_alphas)
            self.z = self.z + alpha * direction

    def find_optimal_vertex(self):
        """Find the optimal vertex for the objective function and return the cost."""
        perturb_count = 0
        while True:
            try:
                self.find_initial_vertex()  # Find initial vertex or restart from degenerate vertex
                vertex_tuple = (tuple(np.round(self.z, decimals=10)), np.dot(self.c, self.z).item())
                if vertex_tuple in self.visited_set:
                    raise ValueError("Cycling detected. Terminating algorithm.")
                self.visited_vertices.append(vertex_tuple)
                self.visited_set.add(vertex_tuple)

                while True:
                    self.__activeAndUnActiveConst()
                    A1 = self.A1

                    if A1.shape[0] != self.n:
                        raise ValueError("Active constraint matrix A1 must be square.")

                    try:
                        # Use pseudo-inverse for singular cases
                        A1_inv = np.linalg.pinv(A1)
                    except np.linalg.LinAlgError:
                        raise ValueError("Active constraint matrix A1 is singular.")

                    beta = np.dot(A1_inv.T, self.c)
                    if np.all(beta >= -self.tolerance):
                        final_cost = np.dot(self.c, self.z).item()
                        return self.visited_vertices

                    # Implementing Bland's Rule for choosing entering variable
                    # Select the smallest index with negative reduced cost
                    entering_indices = np.where(beta < -self.tolerance)[0]
                    if entering_indices.size == 0:
                        final_cost = np.dot(self.c, self.z).item()
                        return self.visited_vertices
                    entering_var = entering_indices[0]  # Bland's Rule: choose the smallest index

                    # Calculate direction to move
                    u = -A1_inv[entering_var].flatten()

                    # Calculate allowable step size (alpha)
                    temp = np.dot(self.A2, u)
                    temp = np.where(np.abs(temp) < self.tolerance, self.tolerance, temp)
                    alpha_values = (self.B2 - np.dot(self.A2, self.z)) / temp
                    positive_alphas = [alpha for alpha in alpha_values if alpha > self.tolerance]

                    if not positive_alphas:
                        raise ValueError("Problem is unbounded.")

                    alpha = min(positive_alphas)
                    self.z = self.z + alpha * u

                    # Round z to mitigate floating-point precision issues
                    self.z = np.round(self.z, decimals=10)

                    # Check for cycling
                    vertex_tuple = (tuple(np.round(self.z, decimals=10)), np.dot(self.c, self.z).item())
                    if vertex_tuple in self.visited_set:
                        raise ValueError("Cycling detected. Terminating algorithm.")
                    self.visited_vertices.append(vertex_tuple)
                    self.visited_set.add(vertex_tuple)

            except ValueError as e:
                # Perturb b again and retry if a degenerate or cycling case is detected
                if perturb_count >= self.max_iterations:
                    raise ValueError("Max perturbations reached without resolving degeneracy or cycling.")
                perturb_count += 1
                print(f"Degenerate or cycling case detected: {str(e)}. Reapplying perturbation.")
                self.perturb_b()
                # Update the perturbation scale incrementally to ensure convergence
                self.epsilon *= 10  # Increase the scale to ensure new perturbations are distinct
                # Reset visited vertices to avoid false cycling due to perturbation
                self.visited_vertices = []
                self.visited_set = set()
                # Optionally, reset z to a known feasible point or maintain the current z

    def get_solution(self):
        """Retrieve the optimal solution and its cost."""
        vertices = self.find_optimal_vertex()
        optimal_vertex, optimal_cost = vertices[-1]
        return optimal_vertex, optimal_cost, vertices


In [41]:
# Example of usage:
# Create a CSV file with the required input format and pass its path to the SimplexAlgo class.
simplex = SimplexAlgo(r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv')
result = simplex.find_optimal_vertex()


Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while finding initial vertex.. Reapplying perturbation.
Degenerate or cycling case detected: Max iterations reached while findin

KeyboardInterrupt: 

In [40]:
result

[((-101.01010102, 90.89890102), -60.56530101999999),
 ((990.889801001, -1001.001001001), 1031.3346010009996)]

In [42]:
import numpy as np
import pandas as pd
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        """
        Initialize the Simplex Algorithm with data from a CSV file.

        Args:
            filePath (str): Path to the CSV file containing the simplex tableau.
        """
        # Read the CSV file without headers
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)
        
        # Extract vectors and matrix based on the input format
        self.z = arr.iloc[0, :-1].values.astype(np.float64)  # Initial feasible point (shape: (n,))
        self.c = arr.iloc[1, :-1].values.astype(np.float64)  # Cost vector (shape: (n,))
        self.b = arr.iloc[2:, -1].values.astype(np.float64)  # Constraint bounds (shape: (m,))
        self.A = arr.iloc[2:, :-1].values.astype(np.float64)  # Constraint matrix (shape: (m, n))
        
        self.n = self.A.shape[1]  # Number of variables
        self.m = self.A.shape[0]  # Number of constraints
        
        if self.m < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")
        
        self.b_original = self.b.copy()  # Save the original b for perturbation
        self.epsilon = 1e-5  # Initial perturbation factor to handle degeneracy

    def isFeasible(self, z=None):
        """
        Check if the current solution z is feasible.

        Args:
            z (np.ndarray, optional): Solution vector to check. Defaults to self.z.

        Returns:
            bool: True if feasible, False otherwise.
        """
        if z is None:
            z = self.z
        return np.all(np.matmul(self.A, z) <= self.b + 1e-8)  # Added tolerance for numerical stability

    def __classify_constraints(self, z):
        """
        Classify constraints as active or inactive based on the current z.

        Args:
            z (np.ndarray): Current solution vector.
        """
        constraint_values = np.matmul(self.A, z)
        tolerance = 1e-6  # Tolerance for determining active constraints
        active_indices = np.where(np.abs(constraint_values - self.b) <= tolerance)[0]
        inactive_indices = np.where(constraint_values < self.b - tolerance)[0]
        
        self.A1 = self.A[active_indices]
        self.b1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.b2 = self.b[inactive_indices]
        self.active_indices = active_indices
        self.inactive_indices = inactive_indices
        
        print(f"Classified {len(active_indices)} active constraints and {len(inactive_indices)} inactive constraints.")

    def __isVertex(self, z):
        """
        Check if the current solution z is a vertex.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if z is a vertex, False otherwise.
        """
        self.__classify_constraints(z)
        
        if self.A1.shape[0] < self.n:
            print("Not enough active constraints to form a vertex.")
            return False
        if self.A1.shape[0] > self.n:
            # More active constraints than variables may indicate degeneracy
            print("More active constraints than variables; potential degeneracy.")
            return False
        
        # Check if the active constraints matrix has full rank
        rank = np.linalg.matrix_rank(self.A1)
        print(f"Rank of active constraints: {rank} (required: {self.n})")
        if rank < self.n:
            # Active constraints do not form a full rank matrix
            print("Active constraints do not form a full rank matrix.")
            return False
        
        return True

    def __testDegeneracy(self, z):
        """
        Test if the current vertex is degenerate.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if degenerate, False otherwise.
        """
        self.__classify_constraints(z)
        rank = np.linalg.matrix_rank(self.A1)
        is_degenerate = rank < self.A1.shape[0]
        print(f"Testing degeneracy: Rank={rank}, Number of active constraints={self.A1.shape[0]} -> {'Degenerate' if is_degenerate else 'Non-degenerate'}")
        return is_degenerate

    def __removeDegeneracy(self):
        """
        Perturb b slightly to handle degeneracy by adding a small epsilon to active constraints.
        """
        perturbation = self.epsilon ** np.arange(1, len(self.active_indices)+1)
        self.b[self.active_indices] = self.b_original[self.active_indices] + perturbation
        print(f"Perturbing b for active constraints with epsilon={self.epsilon}: {self.b[self.active_indices]}")
        self.epsilon *= 2  # Increase epsilon for future perturbations if needed

    def __moveToVertex(self, z):
        """
        Move the current solution z to a valid vertex by considering directions in the nullspace.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            np.ndarray: Updated solution vector after moving towards a vertex.
        """
        while not self.__isVertex(z):
            self.__classify_constraints(z)
            print(f"Active Constraints: {self.A1.shape}, Inactive Constraints: {self.A2.shape}")

            if self.A1.size == 0:
                print("No active constraints to form a vertex. Selecting all variables to move.")
                nullspace = np.eye(self.n)
            else:
                nullspace = np.array(Matrix(self.A1).nullspace())
                nullspace = nullspace.astype(np.float64)
            
            if nullspace.size == 0:
                print("Already at a vertex.")
                break  # Already at a vertex

            # Iterate through possible directions in the nullspace
            for ns_vec in nullspace:
                for direction in [ns_vec, -ns_vec]:
                    direction = direction.flatten()
                    if np.linalg.norm(direction) == 0:
                        continue
                    direction = direction / np.linalg.norm(direction)  # Normalize direction

                    print(f"Trying direction: {direction}")

                    # Compute potential step sizes (alpha)
                    temp = np.dot(self.A2, direction)
                    valid_indices = temp > 1e-8  # Only consider constraints that allow movement in this direction

                    if not np.any(valid_indices):
                        continue  # No valid movement in this direction

                    alpha = (self.b2 - np.dot(self.A2, z)) / temp
                    alpha = alpha[valid_indices]
                    alpha = alpha[alpha > 1e-8]

                    if alpha.size == 0:
                        continue  # No valid positive alpha found

                    alpha_min = np.min(alpha)
                    print(f"Found alpha={alpha_min} for direction={direction}")

                    # Update z
                    z_new = z + alpha_min * direction
                    print(f"Moving to new z: {z_new}")
                    return z_new

            # If no valid direction found, declare no movement possible
            raise ValueError("No valid direction found to move towards a vertex.")

        return z

    def __testOptimum(self, z):
        """
        Test if the current vertex is optimal.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if optimal, False otherwise.
        """
        self.__classify_constraints(z)

        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            raise ValueError("Active constraint matrix A1 is singular.")

        beta = np.dot(A1_inv.T, self.c)
        print(f"Beta values: {beta}")

        is_optimal = np.all(beta >= -1e-6)  # Using a small tolerance for numerical stability
        print(f"Testing optimality: {'Optimal' if is_optimal else 'Not optimal'}")
        return is_optimal

    def __optimize(self, z):
        """
        Optimize from the current vertex to find the next optimal vertex.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            tuple: (Updated solution vector, True if optimization should continue, False otherwise)
        """
        self.__classify_constraints(z)

        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            raise ValueError("Active constraint matrix A1 is singular.")

        beta = np.dot(A1_inv.T, self.c)
        print(f"Beta values for optimization: {beta}")

        # Check if all beta >= 0 (with tolerance)
        if np.all(beta >= -1e-6):
            # Current vertex is optimal
            final_cost = np.dot(self.c, z).item()
            print(f"Optimal vertex found: Vertex {z} with cost {final_cost}")
            return z, False  # No further optimization needed

        # Find all variables with negative beta to consider entering
        entering_vars = [i for i, val in enumerate(beta) if val < -1e-6]

        if not entering_vars:
            # No variables to enter; already optimal
            return z, False

        # Iterate over all entering variables to find a feasible direction
        for entering_var_index in entering_vars:
            # Determine the direction to move
            direction = A1_inv[:, entering_var_index]
            direction = direction.flatten()
            print(f"Attempting to enter variable index: {entering_var_index}, Direction: {direction}")

            # Compute potential step sizes (alpha)
            temp = np.dot(self.A2, direction)
            valid_indices = temp > 1e-8  # Only consider constraints that allow movement in this direction

            if not np.any(valid_indices):
                # Cannot move in this direction without violating constraints
                print(f"No feasible movement in direction for variable index {entering_var_index}. Continuing to next entering variable.")
                continue  # Try next entering variable

            alpha = (self.b2 - np.dot(self.A2, z)) / temp
            alpha = alpha[valid_indices]
            alpha = alpha[alpha > 1e-8]

            if alpha.size == 0:
                print(f"No positive alpha found for variable index {entering_var_index}. Continuing to next entering variable.")
                continue  # Try next entering variable

            alpha_min = np.min(alpha)
            print(f"Optimizing along direction with alpha={alpha_min} for variable index {entering_var_index}")

            # Update z
            z_new = z + alpha_min * direction
            print(f"Updated z after optimization: {z_new}")

            return z_new, True  # Continue optimization

        # After checking all entering variables, if no feasible movement found, declare optimal
        print("No feasible movement found for any entering variable. Declaring current vertex as optimal.")
        return z, False

    def solve(self):
        """
        Solve the linear programming problem using the simplex algorithm.
        """
        # Step 1: Check for feasibility of the Initial Point
        if not self.isFeasible():
            print(f"Initial point {self.z} is not feasible...!!!")
            return

        solved = False
        iteration = 0
        max_iterations = 1000  # Prevent infinite loops

        current_z = self.z.copy()

        while not solved and iteration < max_iterations:
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")

            # Step 2: Move towards the first vertex
            try:
                current_z = self.__moveToVertex(current_z)
            except ValueError as e:
                print(f"Error during moving to vertex: {e}")
                return

            # Test for degeneracy of the vertex
            if self.__testDegeneracy(current_z):
                print("Degenerate vertex detected. Applying perturbation...")
                self.__removeDegeneracy()
                continue  # Re-evaluate feasibility and vertex status after perturbation

            # Print the value of the Cost function at this vertex
            current_cost = np.dot(current_z, self.c)
            print(f"Vertex {current_z} --> Cost: {current_cost}")

            # Step 3: Optimize to find the next optimal vertex
            while True:
                is_optimal = self.__testOptimum(current_z)
                if is_optimal:
                    print(f"Optimal solution found: Vertex {current_z} --> Cost: {current_cost}")
                    solved = True
                    break

                # Optimize to find the next vertex
                try:
                    new_z, continue_opt = self.__optimize(current_z)
                except ValueError as e:
                    print(f"Error during optimization: {e}")
                    return

                if not continue_opt:
                    print(f"Optimal solution found: Vertex {new_z} --> Cost: {np.dot(new_z, self.c)}")
                    solved = True
                    break

                # Update current_z for the next iteration
                current_z = new_z
                current_cost = np.dot(current_z, self.c)
                print(f"Vertex {current_z} --> Cost: {current_cost}")

        if iteration == max_iterations:
            print("Maximum iterations reached. Possible cycling or no feasible solution.")
            return

        # Assign the found optimal solution
        self.z = current_z

        # Display the original solution if possible
        X, y = [], []
        for i in range(len(self.A)):
            if np.abs(np.dot(self.A[i], self.z) - self.b_original[i]) <= 1e-6:
                X.append(self.A[i])
                y.append(self.b_original[i])

        if len(X) == self.n:
            try:
                X = np.array(X)
                y = np.array(y)
                z_original = np.linalg.solve(X, y)
                print(f"Original Solution: Vertex {z_original} --> Cost: {np.dot(z_original, self.c)}")
            except np.linalg.LinAlgError:
                print("Cannot find original solution due to singular matrix.")
        else:
            print("Cannot reconstruct the original solution. Insufficient active constraints.")

        print(f"Final Optimal Solution: Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}")

# Example Usage
if __name__ == "__main__":
    # Replace the file path with the path to your CSV input file
    file_path = r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv'
    simplex = SimplexAlgo(file_path)
    simplex.solve()



--- Iteration 1 ---
Classified 1 active constraints and 3 inactive constraints.
Not enough active constraints to form a vertex.
Classified 1 active constraints and 3 inactive constraints.
Active Constraints: (1, 2), Inactive Constraints: (3, 2)
Trying direction: [0. 1.]
Trying direction: [-0. -1.]
Found alpha=2.0 for direction=[-0. -1.]
Moving to new z: [1. 0.]
Classified 3 active constraints and 1 inactive constraints.
Testing degeneracy: Rank=2, Number of active constraints=3 -> Degenerate
Degenerate vertex detected. Applying perturbation...
Perturbing b for active constraints with epsilon=1e-05: [-9.9999e-01  1.0000e-10  1.0000e+00]

--- Iteration 2 ---
Classified 2 active constraints and 2 inactive constraints.
Rank of active constraints: 2 (required: 2)
Classified 2 active constraints and 2 inactive constraints.
Testing degeneracy: Rank=2, Number of active constraints=2 -> Non-degenerate
Vertex [1. 0.] --> Cost: -3.0
Classified 2 active constraints and 2 inactive constraints.
Bet

C:\Users\gupta\AppData\Local\Temp\ipykernel_44880\3293403922.py:163: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.b2 - np.dot(self.A2, z)) / temp
